In [1]:
#install libraries if necessary (i.e. uncomment and run this code)
# import Pkg
# Pkg.add("JuMP")
# Pkg.add("Ipopt")

In [2]:
# do importing work in this tab
using JuMP
using Ipopt

In [3]:
debug = false

# set up global params
global nc = 2 # number of counties
global ni = 3 # number of industries
global η = 0 # the spill-over effect
global τ = 1 # trade frictions
global ρ = 2.5 # the elasiticity of substitution within industry
global σ = 1.7 # the elasiticity of substitution across industries
global L_H = 1 # total mass of labor at home country
global L_F = 1 # total mass of labor at foreign country
global w_H = 1 # wage at home normalized to 1
global Markup_H = 1/(ρ-1) # markup of firm
global E_H = (Markup_H + 1) * w_H # expenditure

global z_H = Matrix((ones(Float64, ni, nc))) # home productivity
global z_F = Matrix(ones(Float64, ni, 1)) # foreign productivity

company_sizes_H = [1/ni for i=1:ni, c=1:nc] 
#takes on the role of μ_lb and μ_ub below
#represents the size of a company relative to the population in the county, range (0,1)
#@show company_sizes_H #<-- if one would like to debug

# The variables below have solid mathematical reason for being here, but don't seem practical
# to me from a programming perspective. It seems they merely define the relative size of a
# company; this can be represented with 1 number instead of 2. 
# μ_lb = Matrix{Real}([0 0.5; 0 0.5])
# # the entries before the semi-colon is industry 1 for all counties
# μ_ub = Matrix{Real}([0.5 1; 0.5 1])

; #eliminates output from this cell

In [4]:
debug = true #modifies debug for this cell

#Define helper functions

#TODO: ALL NEED TO BE REWRITTEN TO USE VARS AS DEFINED IN THE MODEL
#i.e. I do not want to use the awkward matrix l; it's hard to think about index-wise

#DONE REWRITING
function L_ic_H(i,c)
    return (company_sizes_H[i,c]) * (lhh[i,c] + lhf[i,c])
end

#DONE REWRITING
function E_F()
    return E_H * w_F
end

#DONE REWRITING
function pv_ic_H(i,c)
    return E_H * w_H / (z_H[i,c] * L_ic_H(i,c) ^ η)
end

#DONE REWRITING
function pv_ic_Hx(i,c)
    return E_H * w_H / (z_H[i,c] * L_ic_H(i,c) ^ η)
end

#DONE REWRITING
function pv_if_F(i)
    return E_F() / z_F[i]
end

#DONE REWRITING
function pv_if_Fx(i)
    return  E_F() / z_F[i]
end

#DONE REWRITING
# industry_price_home matrix
function p_i_H(i) # calculate industry price index at home 
    # domestic price aggregation
    H_sum = sum((company_sizes_H[i,c] * pv_ic_H(i,c)^(1-ρ)) for c in 1:nc)
    # foreign price aggregation
    F_sum = τ * pv_if_F(i)^(1-ρ)

    sum = (H_sum + F_sum)^(1/(1-ρ))
    return sum
end

#DONE REWRITING
function p_i_F(i) # calculate industry price index at foreign
    Hx_sum = sum((company_sizes_H[i,c] * (τ * pv_ic_Hx(i,c))^(1-ρ)) for c in 1:nc)
    Fx_sum = (pv_if_Fx(i))^(1-ρ)
    
    sum = (Hx_sum + Fx_sum)^(1/(1-ρ))
    return sum
end

#DONE REWRITING
function p_H() # calculate final good price index at home
    sum = sum((p_i_H(i))^(1-σ) for i in 1:ni)
    return sum^(1/(1-σ))
end

#DONE REWRITING
function p_F() # calculate final good price index at foreign
    sum = sum((p_i_F(i))^(1-σ) for i in 1:ni)
    return sum^(1/(1-σ))
end

#DONE REWRITING
function yv_ic_H(i,c)
    return pv_ic_H(i,c)^(-ρ) * p_i_H(i)^(ρ-σ) * (p_H()^(σ-1)) * E_H
end

#DONE REWRITING
function yv_ic_Hx(i,c)
    return pv_ic_Hx(i,c)^(-ρ) * p_i_F(i)^(ρ-σ) * (p_F()^(σ-1)) * E_F()
end

#DONE REWRITING
function yv_if_F(i)
    return pv_if_F(i)^(-ρ) * p_i_H(i)^(ρ-σ) * p_H()^(σ-1) * E_H
end

#DONE REWRITING
function yv_if_Fx(i)
    return pv_if_Fx(i)^(-ρ) * p_i_F(i)^(ρ-σ) * p_F()^(σ-1) * E_F()
end

#DONE REWRITING
# balanced Trade
function exports()
    # Hx_sum = 0
    # Hx_i_sum = 0
    # for i in 1:ni
    #     Hx_i_sum = 0
    #     for j in 1:nc
    #         Hx_i_sum += (μ_ub[i,j] - μ_lb[i,j]) * (τ * pv_ic_Hx(i,j,l)) * (yv_ic_Hx(i,j,l))
    #     end
    #     Hx_sum += Hx_i_sum
    # end

    Hx_sum = sum((company_sizes_H[i,c]) * (τ * pv_ic_Hx(i,c)) * (yv_ic_Hx(i,c)) for i in 1:ni, c in 1:nc)
    return Hx_sum
end

#TODO: rewrite into imports()
function imports()
    # F_sum = 0
    # for i in 1:ni
    #     F_sum +=  (τ * pv_if_F(i,l)) * (yv_if_F(i,l))
    # end

    F_sum = sum((τ * pv_if_F(i)) * (yv_if_F(i)) for i in 1:ni)
    return F_sum
end

#Returns the difference between the model's current guess and predicted values
function model_difference()
    #TODO: current version of this function, mirror diff of prev work (no endogenous prod)
    diff = 0.0

    for i=1:ni
        for c=1:nc
            #only calculate once, save compute power
            productivity_labor_units = z_H[i,c] * L_ic_H(i,c) ^ η

            #calc diff for lhh_ic
            pred_lhh_ic = yv_ic_H(i,c) / productivity_labor_units
            diff_lhh_ic = pred_lhh_ic - lhh[i,c]

            #calc diff for lhf_ic
            pred_lhf_ic = yv_ic_Hx(i,c) / productivity_labor_units
            diff_lhf_ic = pred_lhf_ic - lhf[i,c]

            diff += diff_lhh_ic^2 + diff_lhf_ic^2
        end
        #calc diff for lfh_i
        pred_lfh_i = τ * yv_if_F(i) / z_F[i]
        diff_lfh_i = pred_lfh_i - lfh[i]

        #calc diff for lff_i
        pred_lff_i = τ * yv_if_Fx(i) / z_F[i]
        diff_lff_i = pred_lff_i - lff[i]

        diff += diff_lfh_i^2 + diff_lff_i^2
    end

    return diff
end

#<<FOR TESTING PURPOSES>>
#TODO: REMOVE/COMMENT OUT ONCE NO LONGER NEEDED FOR TESTING
function model_difference_test()
    diff_lhh = sum((lhh[i,j])^2 for i=1:ni, j=1:nc)
    diff_lff = sum((lff[i])^2 for i=1:ni)
    return 0.0
    return diff_lhh+diff_lff
end

debug = false #resets debug to false after this cell
;

In [5]:
#MAKE JuMP MODEL

debug = false #changes debug for this cell

#Make model and its vars
model = Model(Ipopt.Optimizer)
register(model, :pv_ic_H, 2, pv_ic_H; autodiff = true)
register(model, :pv_ic_Hx, 2, pv_ic_Hx; autodiff = true)
register(model, :p_i_H, 1, p_i_H; autodiff = true)
register(model, :p_i_F, 1, p_i_F; autodiff = true)
register(model, :p_H, 0, p_H; autodiff = true)
register(model, :p_F, 0, p_F; autodiff = true)
register(model, :yv_ic_H, 2, yv_ic_H; autodiff = true)
register(model, :yv_ic_Hx, 2, yv_ic_Hx; autodiff = true)
register(model, :yv_if_F, 1, yv_if_F; autodiff = true)
register(model, :yv_if_Fx, 1, yv_if_Fx; autodiff = true)
register(model, :imports, 0, imports; autodiff = true)
register(model, :exports, 0, exports; autodiff = true)
register(model, :model_difference_test, 0, model_difference_test; autodiff = true)
set_silent(model)
industries = 1:ni
counties = 1:nc

#labor for each industry-county (home production for home consumption), denoted lhh
@variable(model, lhh[industries, counties] >= 0)

#labor for each industry-county (home production for foreign consumption), denoted lhf
@variable(model, lhf[industries, counties] >= 0)

#labor for each industry (foreign production for home consumption), denoted lfh
@variable(model, lfh[industries] >= 0)

#labor for each industry (foreign production for foreign consumption), denoted lff
@variable(model, lff[industries] >= 0)

#TODO: make variable for foreign wage? It seems a fixed point is needed for it as well
@variable(model, w_F >= 0)
#This will also remove some of the ugly index arithmetic in earlier work

#show what these look like, purely for debugging
if(debug)
    @show lhh
    @show lhf
    @show lfh
    @show lff
end

#Set constraints and objective function for our model

#Constraint: labor must never be negative. This is achieved in the variable declarations.

#Constraint: labors in a country must add to the country's initial allotment of labor
@constraint(model, sum(lhh)+sum(lhf) == L_H)
@constraint(model, sum(lfh)+sum(lff) == L_F)

#Constraint: imports must equal predicted exports (or the other way around, it doesn't matter)
@NLconstraint(model, imports() == exports())

#Objective function: minimize(| x - predicted(x) |) <-- i.e. we attempt to reach a fixed-point solution
@NLobjective(model, Min, model_difference_test())
#@objective(model, Min, model_difference(lhh, lhf, lfh, lff))
;

In [6]:
# Run model with newton-krylov
optimize!(model)
@show solution_summary(model)
@show value.(lhh)
@show value.(lhf)
@show value.(lfh)
@show value.(lff)
@show value.(w_F)

; #eliminate unnecessary output


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



LoadError: /(::Float64,::QuadExpr) is not defined. Are you trying to build a nonlinear problem? Make sure you use @NLconstraint/@NLobjective.